# Experimentos y graficos usados para el tp



In [1]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build/ && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../cmake-build-debug/ && make install


mkdir: no se puede crear el directorio «build»: El archivo ya existe
/bin/bash: cmake: orden no encontrada
[ 50%] Built target metnum
[100%] Built target tp2
Install the project...
-- Install configuration: "Debug"
-- Up-to-date: /home/mariana/Facultad/Metodos/TP2/tp2_metodos/notebooks/metnum.cpython-36m-x86_64-linux-gnu.so


# Importamos todo lo que haga falta

In [43]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import metnum

from sklearn.metrics import average_precision_score, accuracy_score, f1_score, precision_score,recall_score, jaccard_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import numpy as np
import pandas as pd
import time
import seaborn as sns


/home/mariana/Facultad/Metodos/TP2/tp2_metodos/notebooks
Python 3.6.9


# Levantamos los archivos de train y test

In [88]:
%load_ext autoreload
%autoreload 2


df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
print(f"Shape train: {df_train.shape}, Shape test: {df_test.shape}")
#df_train=df_train[:10000]

# Uso values para mandar todo a arrays de numpy
X_train = df_train[df_train.columns[1:]].values
y_train = df_train["label"].values.reshape(-1, 1)
# X_test=df_test.values
# y_test= df_labels_test["Label"].values.reshape(-1, 1)


limit = int(0.8 * X_train.shape[0]) 

X_tr, y_tr = X_train[:limit], y_train[:limit]
X_val, y_val = X_train[limit:], y_train[limit:]

assert len(X_tr) == len(y_tr)
assert len(X_val) == len(y_val)

print(f"Ahora tengo {len(X_tr)} instancias de entrenamiento y {len(X_val)} de validación")


train_8000  = (df_train.values)[:8000,1:].reshape(8000,-1)
val_2000    = (df_train.values)[8000:10000,1:].reshape(2000,-1)
labels_2000 = (df_train.values)[8000:10000,0].reshape(2000, 1)
labels_8000 = (df_train.values)[:8000,0].reshape(8000, 1)
print(f"Shape train_8000: {train_8000.shape}")
print(f"Shape labels_8000: {labels_8000.shape}")
print(f"Shape val_2000: {val_2000.shape}")
print(f"Shape labels_2000: {labels_2000.shape}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Shape train: (42000, 785), Shape test: (28000, 784)
Ahora tengo 33600 instancias de entrenamiento y 8400 de validación
Shape train_8000: (8000, 784)
Shape labels_8000: (8000, 1)
Shape val_2000: (2000, 784)
Shape labels_2000: (2000, 1)


In [2]:
%%time
clf=metnum.KNNClassifier(3)
pca=metnum.PCA(19)
pca.fit(X_tr)
X_tr_PCA=pca.transform(X_tr)
X_val_PCA=pca.transform(X_val)

In [3]:
%%time
clf.fit(X_tr_PCA, y_tr)
Y_pred_PCA=clf.predict(X_val_PCA)

### Calculamos la cantidad de imágenes por clase y graficamos

In [4]:
df_train["label"].value_counts()

In [48]:
valores=[4132,4684,4177, 4351,4072,3795,4137, 4401,4063,4188] #Cantidad de datos de las clases ordenadas.
clases=np.arange(10)                                          

prom=np.array(valores).mean()                                 #Calculamos el promedio de elementos por clase.
promedios=[]
for i in range(10):                                           #Ponemos el promedio en un array de 10 posiciones.
    promedios.append(prom)

fig,ax=plt.subplots(figsize=(10,10))
plt.xticks(clases, clases)
plt.xticks(size=12)
plt.yticks(size=12)
plt.bar(x=clases, height=valores, color=colores)
plt.title('Cantidad de elementos por clase', fontsize=16)
for i in range(len(valores)):
    plt.text(x = clases[i]-0.3 , y = valores[i]+0.01, s = str(valores[i]), size = 12)    #Agregamos los valores sobre las barras
plt.xlabel('Cantidad de imágenes', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.ylim(0,5000)
plt.plot(clases,promedios, label='Promedio', color='red')
plt.show()

### Cálculo del accuracy de KNN

In [18]:
%%time
def acc_KNN(x_training, y_training, x_val, y_valid, k):
    clf = metnum.KNNClassifier(k)
    clf.fit(x_training, y_training)
    predicciones=clf.predict(x_val)
    return accuracy_score(y_valid, predicciones)

### Cálculo del accuracy de KNN con PCA

In [49]:
%%time
def acc_KNN_PCA(x_training, y_training, x_val, y_valid, k, alfa):
    clf = metnum.KNNClassifier(k)
    pca=metnum.PCA(alfa)
    pca.fit(x_training)
    x_tr_pca=pca.transform(x_training)
    x_val_pca=pca.transform(x_val)
    
    clf.fit(x_tr_pca, y_training)
    predicciones=clf.predict(x_val_pca)
    return accuracy_score(y_valid, predicciones)

### Cálculo de la accuracy para distintos valores de k

In [50]:
%%time
def plotear_todos_k(save=False):
    df_all_neighbors = pd.read_csv("../data_mediciones/all_predictions.csv") #Leemos el csv que tiene las predicciones para cada k
    all_neighbors_array = (df_all_neighbors.values)[:,1:].astype('int')
    accuracy = [0]
    for i in range(0, all_neighbors_array.shape[1]):
        accuracy.append(accuracy_score(all_neighbors_array[:,i], labels_2000))
    
    
    fig, ax = plt.subplots(figsize=(22,14), dpi=150)
    plt.plot(np.arange(0, 8001), np.array(accuracy), label='Accuracy')
    plt.xscale('log')
    plt.title("Accuracy por k con 8000 imagenes", fontsize=24)
    plt.xlabel('k', fontsize=22)
    plt.ylabel('Accuracy', fontsize=22)
    plt.xlim(1, 8000)
    ax.grid(axis='x', which='both')
    ax.tick_params(which='minor', length=6, width=2)
    plt.legend(loc="upper right", fontsize=19)
    ax.tick_params(labelsize=19)
    
    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_major_formatter(ScalarFormatter())
    
    plt.tight_layout()
    
    if save:
        plt.savefig('../imagenes_informe/accuracy_por_k_8000_imagenes.png')
    plt.show()

### Gráfico del tiempo de ejecución de KNN versus k

In [51]:
%%time
def plotear_ks_vs_tiempo(ks, tiempos, save=False):
    df_ks = pd.read_csv('../data_mediciones/analisis_temporal_ks.csv')
    ks = np.arange(1, 101).tolist()
    acc_ks = df_ks['accuracy'].values
    time_ks = df_ks['time'].values
    ks = np.arange(1, 101).tolist()
    acc_ks, times_ks = acc_x_k(ks, train_8000, labels_8000, val_2000, labels_2000)
    dic_ks = {'accuracy': np.array(acc_ks),
               'time':    np.array(times_ks)
             }
    df_ks = pd.DataFrame(dic_ks)
    df_ks.to_csv('../data_mediciones/analisis_temporal_ks.csv')
    df_ks = pd.read_csv('../data_mediciones/analisis_temporal_ks.csv')
    ks = np.arange(1, 101).tolist()
    acc_ks = df_ks['accuracy'].values
    time_ks = df_ks['time'].values
    
    TITLE_SIZE = 24
    TICK_SIZE = 22
    
    fig, ax = plt.subplots(figsize=(22,14), dpi=150)
    plt.plot(np.array(ks), tiempos, color='orange', label='Tiempo')
    
    m, b = np.polyfit(np.array(ks), np.array(tiempos), 1)
    plt.plot(np.array(ks), m*np.array(ks) + b, 'b--', label='Aproximación lineal')


    plt.title("Tiempo en entrenar y clasificar para distintos ks", fontsize=24)
    plt.xlabel('k', fontsize=TITLE_SIZE)
    plt.ylabel('Tiempo de ejecución en segundos', fontsize=TITLE_SIZE)
    ax.grid(axis='x', which='both')
    

    ticks = np.append([1], np.arange(0,101,10)[1:])    
    plt.xticks(ticks)
    plt.legend(loc="best", fontsize=TITLE_SIZE)
    ax.tick_params(labelsize=TICK_SIZE)
    
    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_major_formatter(ScalarFormatter())
    

    

    plt.tight_layout()
    
    if save:
        plt.savefig('../ks_vs_tiempo_8000_imagenes_line_predict.png')
    plt.show()

### Ploteo de el accuracy para KNN utilizando datasets de distintos tamaños

Conseguimos las accuracy's por cada dataset 

In [29]:
%%time
def acc_por_n_im(cant_datos, k):                 #Recibe un array con los tamaños de datasets a clasificar
    acc=[]
    tiempos = []
    df_train = pd.read_csv("../data/train.csv")

    for i in tqdm(cant_datos):

        df_train=df_train[:i]

        X = df_train[df_train.columns[1:]].values
        y = df_train["label"].values.reshape(-1, 1)


        limit = int(0.8 * X.shape[0]) 
        X_train, y_train = X[:limit], y[:limit]
        X_val, y_val = X[limit:], y[limit:]

        start = time.time()
        acc.append(acc_KNN(X_train, y_train, X_val, y_val,k))
        end =  time.time()

        tiempos.append(end-start)
    return acc, tiempos

Graficamos

In [30]:
%%time
colores=['#2EFE9A', '#BE81F7','#58FAD0']
def plotear_acc_dif_im(eje_x, eje_y, k_usado):
    x=np.arange(len(eje_x))
    fig,ax=plt.subplots(figsize=(10,10))
    plt.xticks(x,eje_x)
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.bar(x=x, height=eje_y, color=colores)
    plt.title('Accuracy KNN por cantidad de imágenes con k = {}'.format(k_usado), fontsize=16)
    for i in range(len(eje_y)):
        plt.text(x = x[i]-0.3 , y = eje_y[i]+0.01, s = str(eje_y[i]), size = 12)
    plt.xlabel('Cantidad de imágenes', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    
    plt.show()

### Ploteo de accuracy para distintos valores de k

In [33]:
%%time
def plotear_acc_por_k(eje_x, eje_y):
    x=np.arange(len(eje_x))
    fig,ax=plt.subplots(figsize=(10,10))
    plt.xticks(x,eje_x)
    plt.xticks(size=16)
    plt.yticks(size=16)
    plt.ylim(0,1.1)
    plt.bar(x=x, height=eje_y, color=colores)
    plt.title('Accuracy variando k', fontsize=20)
    for i in range(len(eje_y)):
        plt.text(x = x[i]-0.3 , y = eje_y[i]+0.01, s = str(eje_y[i]), size = 16)
    plt.xlabel('K', fontsize=20)
    plt.ylabel('Accuracy', fontsize=20)
    
    plt.show()

### Ploteo del porcentaje de varianza aportado por primeras componentes principales

In [ ]:
%%time
def plotear_aporte_autovalores(autovalores, save=False):
    sumas_parciales = np.zeros(autovalores.shape[0])
    sumas_parciales[0] = autovalores[0]
    for i in range(1, sumas_parciales.shape[0]):
        sumas_parciales[i] = sumas_parciales[i-1] + autovalores[i]
    aportes = [100*(elem/sp)[0] for elem, sp in zip(autovalores, sumas_parciales)]

    fig, ax = plt.subplots(figsize=(12,12), dpi=150)

    plt.plot(np.arange(1,101), aportes)
    plt.ylim(0,100)
    plt.xlim(1,100)
    plt.title('Porcentaje de varianza aportado por\nlas primeras 100 componentes principales', fontsize=22)
    plt.xlabel('Alfa', fontsize=22)
    plt.ylabel('Porcentaje de aporte', fontsize=22)
    ax.tick_params(labelsize=20)
    plt.tight_layout()
    
    if save:
        plt.savefig('../imagenes_informe/aporte_autovalores.png')
    
    plt.show()

### Ploteo del accuracy variando con k=5 y 10000 imágenes

In [35]:
%%time
df_all_alfas = pd.read_csv("../data_mediciones/pred_A1_2_A100_n10000.csv") #Levantamos el csv que contiene la data reducida a 100 componentes principales
all_alfas_array = (df_all_alfas.values)[:,1:].astype('int')
print(f"Cantidad de vecinos distintos probados: {all_alfas_array.shape[1]}")
print(f"Tamaño del dataset de testing         : {all_alfas_array.shape[0]}")

accuracy_a = [0]
for i in range(0, all_alfas_array.shape[1]):
    accuracy_a.append(accuracy_score(all_alfas_array[:,i], labels_2000))


def plotear_todos_alfa(acc, save=False):
    TITLE_SIZE = 24
    TICK_SIZE = 22
    
    fig, ax = plt.subplots(figsize=(22,14), dpi=150)
    plt.plot(np.arange(0, len(acc)), np.array(acc), label='Accuracy')
    
    plt.title("Accuracy por alfa con 8000 imagenes y k=5", fontsize=24)
    plt.xlabel('Alfa', fontsize=TITLE_SIZE)
    plt.ylabel('Accuracy', fontsize=TITLE_SIZE)
    plt.xlim(0, len(acc)-1)
    ax.grid(axis='x', which='both')
    plt.xticks(np.arange(0,101,10))
    plt.legend(loc="best", fontsize=TITLE_SIZE)
    ax.tick_params(labelsize=TICK_SIZE)
    
    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_major_formatter(ScalarFormatter())
    
    plt.tight_layout()
    
    if save:
        plt.savefig('../imagenes_informe/accuracy_por_alfa_8000_imagenes.png')
    plt.show()
    
plotear_todos_alfa(accuracy_a)

### Análisis temporal de alfa vs tiempo

In [37]:
%%time
def plotear_alfas_vs_tiempo(alfas, tiempos, save=False):
    TITLE_SIZE = 24
    TICK_SIZE = 22
    
    fig, ax = plt.subplots(figsize=(22,14), dpi=150)
    plt.plot(np.array(alfas), tiempos, color='orange', label='Tiempo')
    
    m, b = np.polyfit(np.array(alfas), np.array(tiempos), 1)
    print(f"m: {m}, b: {b}")
    plt.plot(np.array(alfas), m*np.array(alfas) + b, 'b--', label='Aproximación lineal')
    
    
    plt.title("Tiempo en entrenar y clasificar para distintos alfas con k=5", fontsize=24)
    plt.xlabel('Alfa', fontsize=TITLE_SIZE)
    plt.ylabel('Tiempo de ejecución en segundos', fontsize=TITLE_SIZE)
    ax.grid(axis='x', which='both')
    plt.xticks(alfas)
    plt.legend(loc="best", fontsize=TITLE_SIZE)
    ax.tick_params(labelsize=TICK_SIZE)
    
    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_major_formatter(ScalarFormatter())
    
    plt.tight_layout()
    
    if save:
        plt.savefig('../imagenes_informe/alfas_vs_tiempo_8000_imagenes_line_predict.png')
    plt.show()
    


### Ploteo del accuracy obtenido con KNN y PCA para distintos alfas y k fijo

In [ ]:
%%time
def plotear_acc_por_alfa(alfas, accs, k_usado):
    x=np.arange(len(alfas))
    fig,ax=plt.subplots(figsize=(12,10))
    plt.xticks(x,alfas)
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.ylim(0.0,1.1)
    plt.bar(x=x, height=accs, color=colores)
    plt.title('Accuracy para distintos alfas con k= {}'.format(k_usado), fontsize=16)
    for i in range(len(accs)):
        plt.text(x = x[i]-0.3 , y = accs[i]+0.01, s = str(accs[i]), size = 12)
    plt.xlabel('Alfa', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    
    plt.show()

### Heatmaps de Cross Validation

In [45]:
%%tiempo
def plotear_heatmap(data, ks, alfas, K, save=False):
    fig, ax = plt.subplots(figsize=(8,6), dpi=150)

    sns.set(font_scale=1.25)
    ax = sns.heatmap(data, cmap='viridis', vmin=0.93, vmax=0.955)

    ax.collections[0].colorbar.set_label("Accuracy", fontsize=20)
    ax.set_xlabel('Alfa', fontsize=20)
    ax.set_xticklabels(alfas, fontsize=20)
    ax.set_yticklabels(ks, fontsize=20)
    ax.set_ylabel('k', fontsize=20)
    ax.set_title(f'Heatmap con el accuracy promedio de cada\npareja luego de cross validation con {K} folds')
    plt.tight_layout()
    
    if save:
        plt.savefig(f'../imagenes_informe/heatmap_{K}_folds.png')
    
    plt.show()

### Matriz de confusión

In [47]:
%%time
from sklearn.metrics import confusion_matrix

def matriz_de_conf(y_val, y_pred):
    cm = confusion_matrix(y_val,y_pred, labels=clases)
    fig,ax=plt.subplots(figsize=(10,10), dpi=150)


    ax=sns.heatmap(cm, cmap='viridis', annot=True,fmt="d")
    ax.collections[0].colorbar.set_label("Cantidad de datos clasificados", fontsize=20)
    plt.title('Matriz de confusión sobre los datos de validación', fontsize=18)
    plt.ylabel('Labels reales', fontsize=16)
    plt.xlabel('Labels predichos', fontsize=16)
    plt.tight_layout()

### Ploteo de los números

In [ ]:
%%time
df_mat_conf = pd.read_csv('../data_mediciones/pred_a19_k3_conf_mat.csv')
df_real = pd.read_csv('../data/train.csv')

imagen_mat_conf = df_real.values[33600:, 1:]
real_mat_conf = df_real.values[33600:,0]
pred_mat_conf = df_mat_conf['alfa: 19'].values
accuracy_score(real_mat_conf, pred_mat_conf)

i_uno_x_cero = []
i_cuatro_x_nueve = []
i_nueve_x_cuatro = []
indice_cero = -1
for i in range(real_mat_conf.shape[0]):
    if real_mat_conf[i] == 0:
        indice_cero = i
    if real_mat_conf[i] == 0 and pred_mat_conf[i] == 1:
        i_uno_x_cero.append(i)
    if real_mat_conf[i] == 4 and pred_mat_conf[i] == 9:
        i_nueve_x_cuatro.append(i)
    if real_mat_conf[i] == 9 and pred_mat_conf[i] == 4:
        i_cuatro_x_nueve.append(i)

i_uno_x_cero = i_uno_x_cero[0]

def plotear_numero(indice, imagenes):
    imagen = imagenes[indice].reshape(28, 28) 
    fig, ax = plt.subplots(figsize=(5,5), dpi=150)
    plt.imshow(imagen, cmap="gray_r")
    plt.grid(False) 
    plt.tight_layout()
    plt.show()

# Cross validation

In [5]:
%%time
import numpy as np
from tqdm import tqdm

def train_val(data,labels, K, fold):
    datos_x_fold= data.shape[0]//K
    train= []
    labels_train=[]
    val=[]
    labels_val=[]
    for i in range(data.shape[0]):
        if i<fold*(datos_x_fold) or i>=(fold+1)*(datos_x_fold):
            train.append(data[i])
            labels_train.append(labels[i])
        else:
            val.append(data[i])
            labels_val.append(labels[i])
    return np.array(train),np.array(labels_train), np.array(val), np.array(labels_val)


def cross_val_KNN(set_train,labels_train, set_test,labels_test, ks, K):
    mejorK=ks[0]
    mejor_prec=-1
    precisiones=[]
    for k in tqdm(range(len(ks))):
    
        precision_por_bloque=0
        for i in range(K):
            X_training, y_training, X_valid, y_valid = train_val(set_train, labels_train, K, i)
            cls = metnum.KNNClassifier(ks[k])   
            cls.fit(X_training, y_training)
            y_pred = cls.predict(X_valid)
        

            acc = accuracy_score(y_valid, y_pred)
            precision_por_bloque+=acc
        
        prec=precision_por_bloque/K
        precisiones.append([prec, ks[k]])
        
        if mejor_prec==-1 or prec>mejor_prec:
                mejor_prec=prec
                mejorK=ks[k]
           
    clf = metnum.KNNClassifier(mejorK)
    clf.fit(set_train, y_train)
    y_pred = clf.predict(set_test)

    acc_final = accuracy_score(labels_test, y_pred)

    return [[acc_final,mejorK], precisiones] #Devolvemos acc testing con mejor K y lista de accuracy por k en validacion
    

   
    
    
def cross_val_PCA(set_train,labels_train,ks, alphas,K):
    mejorK=ks[0]
    mejorA=alphas[0]
    n_comp= max(alphas)
    mejor_prec=-1
    precisiones=np.zeros((len(ks), len(alphas)))
    
    pca=metnum.PCA(n_comp)
    
    
    for i in range(K):
        
        X_training, y_training, X_valid, y_valid = train_val(set_train, labels_train, K, i)
        pca.fit(X_training)
        
        

        for indexK, k in enumerate(ks):
            cls = metnum.KNNClassifier(k)

            for indexA, a in enumerate(alphas):
                
                X_training_PCA=pca.custom_transform(X_training,a)
                X_valid_PCA=pca.custom_transform(X_valid,a)
                
                cls.fit(X_training_PCA, y_training)
                y_pred = cls.predict(X_valid_PCA)


                acc = accuracy_score(y_valid, y_pred)
                precisiones[indexK][indexA]+=acc
                print('acc:', acc, '\n','K: ', k, 'A: ',a)
                print('Precisiones:\n', precisiones)
   
        
    precisiones/=K
    
    maximos=np.where(precisiones == precisiones.max())
    
    mejor_prec=precisiones.max()
    mejorK=ks[maximos[0][0]]
    mejorA=alphas[maximos[1][0]]

    return [[mejor_prec,mejorK, mejorA], precisiones]   




CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 15.5 µs
